In [6]:
# imports
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
import torchvision.models as models
from torchvision.models import MobileNet_V3_Small_Weights

from torchinfo import summary

from introdl.utils import get_device, load_results, load_model, summarizer
from introdl.idlmam import train_network
from introdl.visul import plot_training_metrics

sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = [8, 6]  # Set the default figure size (width, height) in inches

In [14]:
# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# Define transformations with normalization
transform = transforms.Compose([
    transforms.Resize(224),                          # Resize the shortest side to 130
    transforms.CenterCrop(224),                      # Center crop to 128x128
    transforms.ToTensor(),                           # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # Normalize with ImageNet stats
                         std=[0.229, 0.224, 0.225])
])

# Load dataset
dataset_path = "../../data/cats_and_dogs/PetImages"
dataset = ImageFolder(root=dataset_path, transform=transform)

# Split into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [15]:

# Load the pre-trained model with new API
model = models.mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)

# Change the number of output classes to 3
model.classifier[-1] = torch.nn.Linear(model.classifier[-1].in_features, 3)

# Freeze all layers except the classifier
for param in model.features.parameters():
    param.requires_grad = False

In [16]:
summarizer(model, input_size=(16, 3, 128, 128))

Layer (type:depth-idx)                             Input Shape          Output Shape         Param #
MobileNetV3                                        [16, 3, 128, 128]    [16, 3]              --
├─Sequential: 1-1                                  [16, 3, 128, 128]    [16, 576, 4, 4]      --
│    └─Conv2dNormActivation: 2-1                   [16, 3, 128, 128]    [16, 16, 64, 64]     --
│    │    └─Conv2d: 3-1                            [16, 3, 128, 128]    [16, 16, 64, 64]     (432)
│    │    └─BatchNorm2d: 3-2                       [16, 16, 64, 64]     [16, 16, 64, 64]     (32)
│    │    └─Hardswish: 3-3                         [16, 16, 64, 64]     [16, 16, 64, 64]     --
│    └─InvertedResidual: 2-2                       [16, 16, 64, 64]     [16, 16, 32, 32]     --
│    │    └─Sequential: 3-4                        [16, 16, 64, 64]     [16, 16, 32, 32]     (744)
│    └─InvertedResidual: 2-3                       [16, 16, 32, 32]     [16, 24, 16, 16]     --
│    │    └─Sequential: 3-5

In [17]:
loss_func = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.AdamW(model.parameters())  # Adam optimizer

device = get_device()
print(device)

ckpt_file = 'models/model_mnv3.pt'
epochs = 3

score_funcs = {'ACC':accuracy_score}

mps


In [18]:
results = train_network(model,
                        loss_func,
                        train_loader,
                        device=device,
                        test_loader=test_loader,
                        epochs = epochs,
                        optimizer = optimizer,
                        score_funcs = score_funcs,
                        checkpoint_file=ckpt_file,
                        resume_checkpoint=False)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]/Users/jbaggett/miniforge3/envs/DS776v2/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch:  33%|███▎      | 1/3 [01:37<03:14, 97.11s/it]/Users/jbaggett/miniforge3/envs/DS776v2/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch:  67%|██████▋   | 2/3 [03:07<01:33, 93.41s/it]/Users/jbaggett/miniforge3/envs/DS776v2/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch: 100%|██████████| 3/3 [04:38<00:00, 92.87s/it]


In [19]:
results

,epoch,total time,train loss,test loss,train ACC,test ACC
0,0,78.855764,0.200555,0.118011,0.915842,0.9516
1,1,152.592266,0.151223,0.116694,0.937394,0.9540
2,2,226.146204,0.133416,0.127648,0.945745,0.9504


In [20]:
# Unfreeze all layers
for param in model.features.parameters():
    param.requires_grad = True

In [21]:
results = train_network(model,
                        loss_func,
                        train_loader,
                        device=device,
                        test_loader=test_loader,
                        epochs = epochs,
                        optimizer = optimizer,
                        score_funcs = score_funcs,
                        checkpoint_file=ckpt_file,
                        resume_checkpoint=True)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]/Users/jbaggett/miniforge3/envs/DS776v2/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch:  33%|███▎      | 1/3 [02:42<05:24, 162.39s/it]/Users/jbaggett/miniforge3/envs/DS776v2/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch:  67%|██████▋   | 2/3 [05:19<02:39, 159.17s/it]/Users/jbaggett/miniforge3/envs/DS776v2/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch: 100%|██████████| 3/3 [07:56<00:00, 158.86s/it]


In [13]:
results

,epoch,total time,train loss,test loss,train ACC,test ACC
0,0,66.886816,0.343676,0.250197,0.848135,0.8926
1,1,125.506997,0.280904,0.249594,0.878638,0.8952
2,2,183.171795,0.254864,0.236762,0.889689,0.9010
3,3,319.437977,0.231022,0.218773,0.906941,0.9044
4,4,444.265769,0.156086,0.213173,0.938294,0.9130
5,5,571.147056,0.129731,0.196475,0.949545,0.9248
